In [1]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import numpy as np

import os

Using TensorFlow backend.


In [2]:
batch_size = 32
num_classes = 3
epochs = 100
data_augmentation = True
# num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_neuron_classification_trained_model.h5'

In [3]:
# write a function to select x_train y_train x_test y_test
data_x = np.load('image_data_256x256.npy')
data_y = np.load('labels.npy')
data_x = np.swapaxes(np.swapaxes(data_x,1,2),2,3)
data_y = np.array([data_y]).T

from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.2)

# shuffle the data
# select 20% as the test data and 80% as the training data
# np.random.choice(len(data_x), len(data_x) * 0.2, replace=False)

/Users/EdwardTONG/.pyenv/versions/2.7.13/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [5]:
# define the model 
# CNN structure
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [6]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [7]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

Using real-time data augmentation.


/Users/EdwardTONG/.pyenv/versions/2.7.13/lib/python2.7/site-packages/keras/preprocessing/image.py:653: UserWarning: Expected input to be images (as Numpy array) following the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (221, 256, 256, 2) (2 channels).
  ' (' + str(x.shape[self.channel_axis]) + ' channels).')
/Users/EdwardTONG/.pyenv/versions/2.7.13/lib/python2.7/site-packages/keras/preprocessing/image.py:787: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (221, 256, 256, 2) (2 channels).
  ' (' + str(self.x.shape[channels_axis]) + ' channels).')


Epoch 1/100
6/6 [==============================] - 201s - loss: 1.0052 - acc: 0.5759 - val_loss: 0.9756 - val_acc: 0.5714
Epoch 2/100
6/6 [==============================] - 195s - loss: 0.9836 - acc: 0.5885 - val_loss: 0.9773 - val_acc: 0.5714
Epoch 3/100
6/6 [==============================] - 190s - loss: 0.9167 - acc: 0.6300 - val_loss: 0.9727 - val_acc: 0.5714
Epoch 4/100
6/6 [==============================] - 188s - loss: 0.9932 - acc: 0.5764 - val_loss: 0.9807 - val_acc: 0.5714
Epoch 5/100
6/6 [==============================] - 194s - loss: 0.9517 - acc: 0.6048 - val_loss: 0.9718 - val_acc: 0.5714
Epoch 6/100
6/6 [==============================] - 192s - loss: 0.9679 - acc: 0.6063 - val_loss: 0.9709 - val_acc: 0.5714
Epoch 7/100
6/6 [==============================] - 195s - loss: 0.9745 - acc: 0.5864 - val_loss: 0.9821 - val_acc: 0.5714
Epoch 8/100
6/6 [==============================] - 193s - loss: 0.9894 - acc: 0.5593 - val_loss: 0.9839 - val_acc: 0.5714
Epoch 9/100
6/6 [=======

6/6 [==============================] - 115s - loss: 0.6281 - acc: 0.7566 - val_loss: 0.9966 - val_acc: 0.5000
Epoch 68/100
6/6 [==============================] - 115s - loss: 0.7025 - acc: 0.7718 - val_loss: 0.7256 - val_acc: 0.7143
Epoch 69/100
6/6 [==============================] - 127s - loss: 0.6867 - acc: 0.7196 - val_loss: 0.7013 - val_acc: 0.7679
Epoch 70/100
6/6 [==============================] - 112s - loss: 0.7593 - acc: 0.6974 - val_loss: 0.7292 - val_acc: 0.6786
Epoch 71/100
6/6 [==============================] - 116s - loss: 0.7802 - acc: 0.6562 - val_loss: 0.6924 - val_acc: 0.7679
Epoch 72/100
6/6 [==============================] - 114s - loss: 0.6309 - acc: 0.7495 - val_loss: 0.8820 - val_acc: 0.5893
Epoch 73/100
6/6 [==============================] - 119s - loss: 0.6916 - acc: 0.7188 - val_loss: 0.6946 - val_acc: 0.7679
Epoch 74/100
6/6 [==============================] - 118s - loss: 0.7258 - acc: 0.7305 - val_loss: 0.9262 - val_acc: 0.5536
Epoch 75/100
6/6 [===========

In [ ]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])